In [34]:
#   Se importan los paquetes basicos para encontrar el perfil de linea

from DefSD import *
#from astropy.analytic_functions import blackbody
#from astropy.convolution import convolve, Gaussian1DKernel, Box1DKernel



In [35]:
###    Se verifica que existan todos los archivos que se van a ajustar.
#      Se puede omitir para trabajar un sol archivo

resultListDT=Table.read('LIST1/listR.tbl', format='ascii')       #testSDSSFound.txt
for x in range(len(resultListDT['ObjID'])):
    name = resultListDT['ObjID'][x]
    try:
        hdulist = fits.open('LIST1/'+name+'.fits')
    except IOError:
        print "Error opening the file: ", name

In [36]:
#   a table resultListDT relates the name of the objects and the resulting temperature and density
#   for the four couples of species

#resultListDT.tbl
#resultListDT=Table.read('testSDSSFound.txt', format='ascii')
resultListDT=Table.read('LIST1/listR.tbl', format='ascii')       #testSDSSFound.txt
ion = [' [OIII]', ' [SII]', ' [NII]', ' [OII]', ' [NII]', ' [SII]', ' [OIII]', ' [OII]',' [OIII]', ' [ArIV]', ' [NII]', ' [ArIV]']
for i in range(1,7):
    colT= Column(name='Temp_'+str(i)+ion[2*(i-1)], length=len(resultListDT),unit=u.K)
    colD= Column(name='Den_'+str(i)+ion[2*(i-1)+1], length=len(resultListDT),unit=1 / (u.cm**3))
    resultListDT.add_columns([colT,colD])
resultListDT

#   Obtain the lists Spectra, Headers, spectrum variance of the intensity according with a list of galaxy names
#   and a Table Tabla of models for each line for all the spectra

Headers = []
Spectra = []
Spectra2 = []
SpectraCont = []
Lambda = []
Modelos=[]
Tablas=[]
HDUs=[]
TypeFITS=[]
SpectraIvar = []
LINEProfile = []
paramCont=0.4
typeFITS =0
for x in range(len(resultListDT['ObjID'])):
    try:
        name = resultListDT['ObjID'][x]
        z=resultListDT['z'][x]
        isFits = resultListDT['isFits'][x]
        if isFits:
            hdulist = fits.open('LIST1/'+name+'.fits')
            head= hdulist[0].header
            typeFITS=0
            if head['NAXIS'] == 0:
                total= hdulist[1].data
            elif head['NAXIS'] == 1:
                typeFITS=1
            if typeFITS==0:
                spectraFlux = total['flux']
                spectraIvar = total['ivar']
                linlam=np.exp(total['loglam']*np.log(10))          #  ocultar para leer desde archivo
                lambd = linlam/(1+z)
            else:
                spectraFlux = hdulist[0].data #*(1e-19)
                spectraIvar = spectraFlux*0
                if 'CDELT1' in head:
                    lambd = np.linspace(head['CRVAL1'], head['CRVAL1']+head['CDELT1']*(len(spectraFlux)-1), len(spectraFlux))
                elif 'W0' in head:
                    lambd = np.linspace(head['W0'], head['W0']+head['WPC']*(len(spectraFlux)-1), len(spectraFlux))
                else:
                    print "Can't identify FITS format in file: ", str(name)
                    break
                lambd = lambd/(1+z)
        else:
            specTable = Table.read('LIST1/'+str(name)+'.txt', format='ascii')
            spectraFlux = specTable['Intensity'].data #.tolist()
            spectraIvar = spectraFlux*0
            lambd = specTable['Spectral-axis'].data #.tolist()
            lambd = lambd/(1+z)
        orderU = 10**(int(mt.log(np.mean(spectraFlux), 10))-1)

        TypeFITS.append(typeFITS)
        Spectra2.append(spectraFlux)
        print "Object [",x,"]: ", name
        spec =np.array(spectraFlux)
        spec2, lambd = r(spec, lambd, paramCont*orderU)
        
        #gauss_kernel = Gaussian1DKernel(2)
        #smoothed_data_gauss = convolve(spectraFlux, gauss_kernel)
        #spec2 = smoothed_data_gauss

        limResolucion = mt.ceil(lambd[1]-lambd[0])

    # ..............    

        if False: # lambd[0] > 3700.0 :
            print "The object isn't in expected spectrum region (3700 - 6800 Angstrom)"
            ListaLineas=Table.read('LinesPrincipal.txt', format='ascii')
            LS=np.zeros([13,11])

            lineProf = lineProfile(x,spec2, lambd,ListaLineas,Spectra2, limResolucion)
            LINEProfile.append(lineProf)
            # Este ciclo calcula remplaza la región del contínuo que está justo en el perfil de línea, por una recta
            # que es mas preciso al determinar la amplitud del modelo gaussiano
            for ina in range(len(lineProf)):
                lInf = lineProf[ina]['inf']
                lSup = lineProf[ina]['sup']
                lDel = lSup-lInf
                if (lInf-3*lDel) >= 0 :
                    lInf = lInf-3*lDel
                if (lSup+3*lDel) <= len(lambd) :
                    lSup = lSup+3*lDel
                m1, b1 = linModel( lambd[lInf] , spec2[lInf] , lambd[lSup], spec2[lSup])
                lineLambd = lambd[lInf:lSup]
                lineIntCont = lines( lineLambd , m1, b1)
                spec2[lInf:lSup] = lineIntCont
            ModelG1, lineProf, spec2 = finalFit(x, spec2, lambd,ListaLineas,Spectra2, spectraIvar,2000, limResolucion)
    #        ModelL1 = fit2(x, spec2, lambd,ListaLineas,Spectra,2500)
        if lambd[0] < 4100.0:
            # Este ciclo calcula remplaza la región del contínuo que está justo en el perfil de línea, por una recta
            # que es mas preciso al determinar la amplitud del modelo gaussiano
            ListaLineas=Table.read('LinesAll.txt', format='ascii')
            lineProf = lineProfile(x,spec2, lambd,ListaLineas,Spectra2, limResolucion)
            LINEProfile.append(lineProf)
            for ina in range(len(lineProf)):
                lInf = lineProf[ina]['inf']
                lSup = lineProf[ina]['sup']
                lDel = lSup-lInf
                if (lInf-3*lDel) >= 0 :
                    lInf = lInf-3*lDel
                if (lSup+3*lDel) <= len(lambd) :
                    lSup = lSup+3*lDel
                m1, b1 = linModel( lambd[lInf] , spec2[lInf] , lambd[lSup], spec2[lSup])
                lineLambd = lambd[lInf:lSup]
                lineIntCont = lines( lineLambd , m1, b1)
                spec2[lInf:lSup] = lineIntCont
            LS=np.zeros([len(ListaLineas),11])
            ModelG1, lineProf, spec2 = finalFit(x, spec2, lambd,ListaLineas,Spectra2, spectraIvar,2000, limResolucion)
    #        ModelL1 = fit2(x, spec2, lambd,ListaLineas,Spectra,2500)

        Tabla=Table(LS,names=('Amplitude','Wavelength','Sigma','Lambda Vac','Element','Flux','Flux_Err', 'FWHM', 'eqWidth', 'S/N>3', 'NAME'),
                    dtype=('f8','f8','f8','f8','S8','f8','f8','f8','f8','S5', 'S12'))
        Tabla['Lambda Vac']=ListaLineas['LAMBDA VAC ANG']
        Tabla['Element']=ListaLineas['SPECTRUM']
        AA=split_list(ModelG1[0].parameters, wanted_parts=len(ListaLineas))
        AAA=split_list(ModelG1[1].parameters, wanted_parts=2)
    #    AA=split_list(ModelL1[0].parameters, wanted_parts=len(ListaLineas))
    #    AAA=split_list(ModelL1[1].parameters, wanted_parts=2)
        Tabla['NAME']=ListaLineas['NAME']
        for ii in range(len(ListaLineas)):
            if 3*np.mean(spectraIvar[lineProf[0]['inf']:lineProf[0]['sup']]) >= AA[ii][0]:
                Tabla['S/N>3'][ii] = str(False)
            else:
                Tabla['S/N>3'][ii] = str(True)
            if AA[ii][2]<0:
                Tabla['Amplitude'][ii],Tabla['Wavelength'][ii],Tabla['Sigma'][ii]=np.array([AA[ii][0], AA[ii][1], mt.fabs(AA[ii][2])])
            else:
                Tabla['Amplitude'][ii],Tabla['Wavelength'][ii],Tabla['Sigma'][ii]=np.array([AA[ii][0], AA[ii][1], AA[ii][2]])
            if ii==0 or ii==1:
                if AAA[ii][2]<0:
                    Tabla['Amplitude'][ii],Tabla['Wavelength'][ii],Tabla['Sigma'][ii]=np.array([AAA[ii][0], AAA[ii][1], mt.fabs(AAA[ii][2])])
                else:
                    Tabla['Amplitude'][ii],Tabla['Wavelength'][ii],Tabla['Sigma'][ii]=np.array([AAA[ii][0], AAA[ii][1], AAA[ii][2]])
        Modelos.append(ModelG1)
        #    Modelos.append(ModelL1)
        #print Tabla 
        Tablas.append(Tabla)
        SpectraIvar.append(spectraIvar)
        SpectraCont.append(spec2)
        Lambda.append(lambd)
        Spectra.append(spectraFlux)
    except ValueError:
        print "It has not been possible to process the spectrum of: ", name

Object [ 0 ]:  Iras23060+0505SP1
Object [ 1 ]:  3C234SP1


In [39]:
#   plot of the galaxy with index jj observed with respect to the continuum in zero. Two models for the lines required 
%matplotlib
### inline
for jj in [0]:  #  range(len(resultListDT)):
#    plt.errorbar(Lambda[jj],Spectra[jj],SpectraIvar[jj],  c='gray',lw=1,label='Spectrum')
    plt.plot(Lambda[jj],Spectra[jj], lw=0.5, c = 'k', label='Spectra')
    plt.plot(Lambda[jj],SpectraCont[jj], lw=2, c = 'g', label='Spectra')
    lPPP = lineProf
    for iasiao in range(len(lPPP)):
        plt.plot(Lambda[jj][lPPP['inf'][iasiao]:lPPP['sup'][iasiao]],Spectra[jj][lPPP['inf'][iasiao]:lPPP['sup'][iasiao]], lw=2.0, c = 'b')


Using matplotlib backend: Qt4Agg


###     EqWidth ,  FWHM,  FLUX

In [40]:
#   The plot of relative intensity and continuum emission for the galaxy index jj and the wavelenght of all the lines
#   The equivalent width, flux and FWHM for galaxy jj is measured too        
for jj in range(len(Tablas)):
    spe = []
    speCont = []
    for xind in range(len(Spectra[jj])):
        spe.append((Spectra[jj][xind])/(SpectraCont[jj][xind]))
        speCont.append((SpectraCont[jj][xind])/(SpectraCont[jj][xind]))

    lFWHM = []
    eqW = []    ##  lista de valores 
    #  una vez normalizado el espectro observado a la medida del contínuo se integra por el método de Simpson
    for iLineProf in range(len(LINEProfile[jj])):
        try:
            f = FWHM(Lambda[jj][LINEProfile[jj]['inf'][iLineProf]:LINEProfile[jj]['sup'][iLineProf]], Spectra[jj][LINEProfile[jj]['inf'][iLineProf]:LINEProfile[jj]['sup'][iLineProf]])
            if len(f) == 0:
                lFWHM.append(0)
            else:
                lFWHM.append(f[0])
        except ValueError:
            lFWHM.append(0)
        if (LINEProfile[jj]['inf'][iLineProf] == LINEProfile[jj]['sup'][iLineProf]) and (LINEProfile[jj]['sup'][iLineProf] == 0):
            eqW.append(0)
        else:
            sInteg = simps(y=spe[LINEProfile[jj]['inf'][iLineProf]:LINEProfile[jj]['sup'][iLineProf]], x=Lambda[jj][LINEProfile[jj]['inf'][iLineProf]:LINEProfile[jj]['sup'][iLineProf]], even='avg')
            eqW.append(sInteg)
    #   Flux determination for each model in galaxy list
    #   assignment of the FWHM and equivalent width in the Table for the first galaxy
    for i in range(len(Tablas[jj])):
        if Tablas[jj][i]['Amplitude']<=0:
            Tablas[jj]['Flux'][i],Tablas[jj]['Flux_Err'][i]= 0,0
        else:
            Tablas[jj]['Flux'][i],Tablas[jj]['Flux_Err'][i]=quad(gaussian,Tablas[jj][i]['Wavelength']-3*Tablas[jj][i]['Sigma'],Tablas[jj][i]['Wavelength']+3*Tablas[jj][i]['Sigma'],args=(Tablas[jj][i]['Amplitude'],Tablas[jj][i]['Wavelength'],Tablas[jj][i]['Sigma']))
            Tablas[jj]['FWHM'][i],Tablas[jj]['eqWidth'][i] = lFWHM[i], eqW[i]
    Tablas[jj].write('LIST1/resultTables/Tabla'+str(resultListDT['ObjID'][jj])+'.tbl',format='ascii')
    Tablas[jj]['NAME', 'Flux'].write('LIST1/resultTables/TablaPN'+str(resultListDT['ObjID'][jj])+'.dat',format='ascii')
#Tablas[jj]

###     Corrección por atenuación debida al polvo

In [41]:
###    Corrección por atenuación debida al polvo de acuerdo a la ley CCM89 con la razón entre
#      Ha y Hb tomando el valor esperado de 2.86 con R_v = 3.1
#      También se puede utilizar el cociente Hgamma / Hb 


#      Esta parte fue tomada del documento de PyNeb para diagramas diagnósticos de las
#      cinco líneas prohibidas seleccionadas


### General settings
# Setting verbosity level. Enter pn.my_logging? for details
pn.log_.level = 2 # set this to 3 to have more details
# Adopt an extinction law
extinction_law = 'CCM89'

for jj in range(len(Tablas)):
    
    # Define the data file
    obs_data = 'LIST1/resultTables/TablaPN'+str(resultListDT['ObjID'][jj])+'.dat'
    # Define plot title
    title = 'Flux'
    ### Read and deredden observational data
    # define an Observation object and assign it to name ’obs’
    obs = pn.Observation()

    # fill obs with data read from file obs_data, with lines varying across rows and a default per
    obs.readData(obs_data, fileFormat='lines_in_rows', err_default=0.05)
    # deredden data with Cardelli’s law
    obs.extinction.law = extinction_law

    ###  Corrección del cociente Ha / Hb cuyo valor esperado es 2.86 para nebulosas planetarias
    obs.def_EBV(label1="H1r_6563A", label2="H1r_4861A", r_theo=2.86)
    obs.correctData(normWave=4861.)

    ###  Corrección del cociente Hgamma / Hb cuyo valor esperado es 0.47 para nebulosas planetarias
    #obs.def_EBV(label1="H1r_4341A", label2="H1r_4861A", r_theo=0.47)
    #obs.correctData(normWave=4861.)

    obs.correctData()
    ### Include the diagnostics of interest
    # instantiate the Diagnostics class
    diags = pn.Diagnostics()
    # include in diags the relevant line ratios
    diags.addDiag([
    '[NII] 5755/6584',
    '[OII] 3726/3729',
    '[OIII] 4363/5007',
    '[SII] 6731/6716',
    '[ArIV] 4740/4711'
    ])
    
    ext=obs.extinction
    print 'Extinction Law: CCM89'
    print 'Object['+str(jj)+'] : ', str(resultListDT['ObjID'][jj]), 'cHbeta =', ext.cHbeta, ' R_V =',  ext.R_V, ' E(B_V) =',  ext.E_BV
    
    TeN2O2, NeN2O2 = diags.getCrossTemDen('[NII] 5755/6584+', '[OII] 3726/3729', obs=obs)
    TeO3O2, NeO3O2 = diags.getCrossTemDen('[OIII] 4363/5007+', '[OII] 3726/3729', obs=obs)
    TeO3S2, NeO3S2 = diags.getCrossTemDen('[OIII] 4363/5007+', '[SII] 6731/6716', obs=obs)
    TeN2S2, NeN2S2 = diags.getCrossTemDen('[NII] 5755/6584+', '[SII] 6731/6716', obs=obs)
    TeO3Ar4, NeO3Ar4 = diags.getCrossTemDen('[OIII] 4363/5007+', '[ArIV] 4740/4711', obs=obs)
    TeN2Ar4, NeN2Ar4 = diags.getCrossTemDen('[NII] 5755/6584+', '[ArIV] 4740/4711', obs=obs)

    resultListDT['Temp_1 [OIII]'][jj], resultListDT['Den_1 [SII]'][jj] = TeO3S2, NeO3S2
    resultListDT['Temp_3 [NII]'][jj], resultListDT['Den_3 [SII]'][jj] = TeN2S2, NeN2S2
    resultListDT['Temp_2 [NII]'][jj], resultListDT['Den_2 [OII]'][jj] = TeN2O2, NeN2O2
    resultListDT['Temp_4 [OIII]'][jj], resultListDT['Den_4 [OII]'][jj] = TeO3O2, NeO3O2
    resultListDT['Temp_6 [NII]'][jj], resultListDT['Den_6 [ArIV]'][jj] = TeN2Ar4, NeN2Ar4
    resultListDT['Temp_5 [OIII]'][jj], resultListDT['Den_5 [ArIV]'][jj] = TeO3Ar4, NeO3Ar4


Extinction Law: CCM89
Object[0] :  Iras23060+0505SP1 cHbeta = [ 2.05399162]  R_V = 3.1  E(B_V) = [ 1.42286475]
Extinction Law: CCM89
Object[1] :  3C234SP1 cHbeta = [ 0.62415763]  R_V = 3.1  E(B_V) = [ 0.43237367]


###               Electron density and temperature results

In [42]:
resultListDT

ObjID,ra,dec,z,isFits,Temp_1 [OIII],Den_1 [SII],Temp_2 [NII],Den_2 [OII],Temp_3 [NII],Den_3 [SII],Temp_4 [OIII],Den_4 [OII],Temp_5 [OIII],Den_5 [ArIV],Temp_6 [NII],Den_6 [ArIV]
,,,,,K,1 / cm3,K,1 / cm3,K,1 / cm3,K,1 / cm3,K,1 / cm3,K,1 / cm3
string136,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
Iras23060+0505SP1,347.14125,5.35833333333,0.1729254,1,12651.1013964,38.7779653136,10222.5129511,399.787023329,10287.1174427,42.3287027062,12651.1013964,438.707188757,12180.9720528,29415.2062025,8350.67763628,25367.6722696
3C234SP1,150.45625,28.7858333333,0.1848955,1,20609.1805563,3836.28425575,5987.95159814,944.303785544,5927.12420385,2381.94771107,20698.1439729,1655.11494516,20300.8090474,12313.1340853,5719.41743731,7738.17096289


In [26]:
resultListDT.write('LIST1/resultTables/DeTe1.tbl',format='ascii')

In [43]:
###    Grafica del diagnóstico densidad y temperatura para el espectro de indice jj

jj=0

%matplotlib

### General settings
# Setting verbosity level. Enter pn.my_logging? for details
pn.log_.level = 2 # set this to 3 to have more details
# Adopt an extinction law
extinction_law = 'CCM89'

# Define the data file
obs_data = 'LIST1/resultTables/TablaPN'+str(resultListDT['ObjID'][jj])+'.dat'
# Define plot title
title = 'Flux'
### Read and deredden observational data
# define an Observation object and assign it to name ’obs’
obs = pn.Observation()

# fill obs with data read from file obs_data, with lines varying across rows and a default per
obs.readData(obs_data, fileFormat='lines_in_rows', err_default=0.05)
# deredden data with Cardelli’s law
obs.extinction.law = extinction_law

###  Corrección del cociente Ha / Hb cuyo valor esperado es 2.86 para nebulosas planetarias
obs.def_EBV(label1="H1r_6563A", label2="H1r_4861A", r_theo=2.86)
obs.correctData(normWave=4861.)

###  Corrección del cociente Hgamma / Hb cuyo valor esperado es 0.47 para nebulosas planetarias
#obs.def_EBV(label1="H1r_4341A", label2="H1r_4861A", r_theo=0.47)
#obs.correctData(normWave=4861.)

obs.correctData()
### Include the diagnostics of interest
# instantiate the Diagnostics class
diags = pn.Diagnostics()
# include in diags the relevant line ratios
diags.addDiag([
'[NII] 5755/6584',
'[OII] 3726/3729',
'[OIII] 4363/5007',
'[SII] 6731/6716',
'[ArIV] 4740/4711'
])

# Create the emission maps to be compared to the observation data (some overkill here)
emisgrids = pn.getEmisGridDict(atom_list=diags.getUniqueAtoms(), den_max=1e6)
### Plot
# Create the contour plot as the intersection of tem-den emission maps with dereddened line ra
diags.plot(emisgrids, obs)
# Place the title
plt.title(title)

Using matplotlib backend: Qt4Agg


###                     Plots the fitting spectrum normalized of index jj

In [45]:
#   plot of the galaxy with index jj observed with respect to the continuum in zero. Two models for the lines required 

%matplotlib
jj=0
if isFits:
    if TypeFITS == 0:
        print HDUs[jj][0].header['FLAVOR']
        print HDUs[jj][0].header['QUALITY']
        print HDUs[jj][0].header['FBADPIX'] ## %%Malos Pixeles
        print HDUs[jj][2].data['SN_MEDIAN_ALL']
        print HDUs[jj][2].data['SN_MEDIAN']

lambd=Lambda[jj]

plt.figure(figsize=(16, 9))
plt.title('Spectrum Galaxy '+ resultListDT['ObjID'][jj])
plt.xlabel(r'Wavelength (Angstroms $\AA$) ')
if TypeFITS == 0:
    plt.ylabel(r'$f_{\lambda}$ ('+str(head['BUNIT'])+')')
else:
    plt.ylabel(r'$f_{\lambda}$')

plt.errorbar(Lambda[jj],Spectra[jj]-SpectraCont[jj],SpectraIvar[jj],  c='gray',lw=1)
plt.plot(Lambda[jj], Spectra[jj]-SpectraCont[jj], c='k', lw=0.8, label='Spectrum')
plt.plot(Lambda[jj], Modelos[jj][0](Lambda[jj]), c='yellow', lw=1 , label='Model')

plt.legend()

x=np.linspace(Lambda[jj][0],Lambda[jj][-1],len(Lambda[jj]))
for i in range(len(Tablas[jj])):
    plt.axvline(x=Tablas[jj]['Lambda Vac'][i],c='dimgrey',lw=1.2,ls='--')
    plt.text(Tablas[jj]['Lambda Vac'][i] + 0.5, Tablas[jj]['Amplitude'][i] * 1.2,Tablas[jj]['Element'][i])
    if Tablas[jj]['Element'][i] == 'H I':
        plt.plot(x,gaussian(x,Tablas[jj][i]['Amplitude'],Tablas[jj][i]['Wavelength'],Tablas[jj][i]['Sigma']), c='DeepPink')
##        plt.plot(x,lorentzFunc(x,Tablas[jj][i]['Amplitude'],Tablas[jj][i]['Sigma'], Tablas[jj][i]['Wavelength']), c='DeepPink')
    else:
        if Tablas[jj][i]['Sigma']!=0:
            plt.plot(x,gaussian(x,Tablas[jj][i]['Amplitude'],Tablas[jj][i]['Wavelength'],Tablas[jj][i]['Sigma']) , c='Lime' , label='Modelos')
##            plt.plot(x,lorentzFunc(x,Tablas[jj][i]['Amplitude'],Tablas[jj][i]['Sigma'],Tablas[jj][i]['Wavelength']) , c='Lime')

#plt.savefig('F1.eps', format='eps', dpi=1000)


Using matplotlib backend: Qt4Agg
